In [2]:
from PIL import Image, ImageDraw
import random
from multiprocessing import Process
from threading import Thread
import cProfile
import pstats
import io
import sys

NUM_NODES_WIDE = 80
NUM_NODES_HIGH = 45
WIDTH = 500
HEIGHT = 500

total_pixel_in = 0
total_pixel_out = 0

random.seed()

#########################################################################################
##          Draws a polygon
#########################################################################################
def drawPolygon(poly, colour, drw):
    drw.polygon(poly, (int(colour[0]), int(colour[1]),int(colour[2])))

#########################################################################################
##          Draws all the nodes
#########################################################################################
def drawNodeList(nodeList, polyColourDict, drw):
    for x in range(0, NUM_NODES_WIDE - 1):
        for y in range(0, NUM_NODES_HIGH - 1):
            drawPolygon([nodeList[x][y], nodeList[x][y + 1], nodeList[x + 1][y + 1]], polyColourDict[tuple(sorted([(x, y), (x, y + 1), (x + 1, y + 1)], key=lambda x: (x[0], x[1])))], drw)
            drawPolygon([nodeList[x][y], nodeList[x + 1][y], nodeList[x + 1][y + 1]], polyColourDict[tuple(sorted([(x, y), (x + 1, y), (x + 1, y + 1)], key=lambda x: (x[0], x[1])))], drw)

#########################################################################################
##          Checks of points p1 and p2 are the same side of line a->b
#########################################################################################
#We can tell if the two points p1 and p2 are on the same side of the line a->b using cross products. (a->b)x(a->p1) should have the same z direction as (a->b)x(a->p2)
def sameSide(vecAB, crossC, a, p):
    vecAP = [a[0] - p[0], a[1] - p[1]] #get vector a->p2
    crossP= vecAB[0]*vecAP[1] - vecAB[1]*vecAP[0] #z direction of ab x ap2

    if crossP * crossC > 0: #test that the two numbers have the same sign ie the z is in the same direction
        return True
    else:
        return False

#########################################################################################
##          Gets the vectors and cross products of a polygon that don't change for speed improvements
#########################################################################################
def getVecsAndCrosses(nodes):
    vecAB = [nodes[0][0] - nodes[1][0], nodes[0][1] - nodes[1][1]]
    vecAC = [nodes[0][0] - nodes[2][0], nodes[0][1] - nodes[2][1]]
    vecBC = [nodes[1][0] - nodes[2][0], nodes[1][1] - nodes[2][1]]
    vecBA = [nodes[1][0] - nodes[0][0], nodes[1][1] - nodes[0][1]]

    crossC = vecAB[0]*vecAC[1] - vecAB[1]*vecAC[0]
    crossB = vecAC[0]*vecAB[1] - vecAC[1]*vecAB[0]
    crossA = vecBC[0]*vecBA[1] - vecBC[1]*vecBA[0]

    vecs = [vecAB, vecAC, vecBC]
    crosses = [crossA, crossB, crossC]
    
    return (vecs, crosses)

#########################################################################################
##          Checks if a pixels is in a polygon
#########################################################################################
def pixelInPolygon(nodes, vecs, crosses, pixel):
    global total_pixel_in
    global total_pixel_out

    if sameSide(vecs[0], crosses[2], nodes[0], pixel) and sameSide(vecs[1], crosses[1], nodes[0], pixel) and sameSide(vecs[2], crosses[0], nodes[1], pixel):
        total_pixel_in += 1
        return True
    total_pixel_out += 1
    return False

#########################################################################################
##          Gets the smallest box that surrounds a polygon
#########################################################################################
def getPolyBox(poly):
    maxX = 0
    maxY = 0
    minX = WIDTH
    minY = HEIGHT

    for node in poly:
        if node[0] > maxX:
            maxX = node[0]
        if node[0] < minX:
            minX = node[0]

        if node[1] > maxY:
            maxY = node[1]
        if node[1] < minY:
            minY = node[1]
    return minX, maxX, minY, maxY

#########################################################################################
##          Calculates the colour a polygon should be
#########################################################################################
def calculatePolyColour(nodes, pix): #List of tuples of pixel coordinates, image pixels
    maxX = 0
    maxY = 0
    minX = WIDTH
    minY = HEIGHT

    redAverage = 0
    blueAverage = 0
    greenAverage = 0

    pixelList = []
    numPixels = 0

    minX, maxX, minY, maxY = getPolyBox(nodes)
    #print(minX, maxX, minY, maxY)
    vecs, crosses = getVecsAndCrosses(nodes)

    for x in range(int(minX), int(maxX)):
        foundPoint = False
        for y in range(int(minY), int(maxY)):
            if pixelInPolygon(nodes, vecs, crosses, [x, y]):
                foundPoint = True
                pixelList.append([x, y])
                redAverage += pix[x][y][0]
                blueAverage += pix[x][y][1]
                greenAverage += pix[x][y][2]
                numPixels += 1
            elif foundPoint == True:
                break

    if numPixels > 0:
        redAverage /= numPixels
        blueAverage /= numPixels
        greenAverage /= numPixels

    difference = 0
    for pixel in pixelList:
        difference += abs(pix[pixel[0]][pixel[1]][0] - redAverage)
        difference += abs(pix[pixel[0]][pixel[1]][1] - blueAverage)
        difference += abs(pix[pixel[0]][pixel[1]][2] - greenAverage)

#If there are pixels under this polygon scale the difference to allow some small polygons
#If there are no pixels, set the colour of the polygon to the colour of one of its
#corners to stop it from incorrectly just being black
    if numPixels > 0:
        difference /= numPixels**0.7
    else:
        redAverage = pix[int(minX)][int(minY)][0]
        blueAverage = pix[int(minX)][int(minY)][1]
        greenAverage = pix[int(minX)][int(minY)][2]

    return (redAverage, blueAverage, greenAverage, int(difference))

#########################################################################################
##          Returns a list of node references connected to provided node
#########################################################################################
def getPolyList(x, y): #In: grid ref, out: list of tuple of grid references
    return [((x, y), (x - 1, y - 1), (x - 1, y)),
            ((x, y), (x, y - 1), (x - 1, y - 1)),
            ((x, y), (x, y - 1), (x + 1, y)),
            ((x, y), (x + 1, y), (x + 1, y + 1)),
            ((x, y), (x + 1, y + 1), (x, y + 1)),
            ((x, y), (x, y + 1), (x - 1, y))]


#########################################################################################
##          Move a node
#########################################################################################
def moveNode(x, y, nodeList, polyColourDict, pix):
    worstDif = 0
    bestDif = 100000
    worstPoly = ()
    worstRealPoly = ()
    polyList = getPolyList(x, y)
    realPolyList = []
    for poly in polyList:
        realPoly = (nodeList[poly[0][0]][poly[0][1]], nodeList[poly[1][0]][poly[1][1]], nodeList[poly[2][0]][poly[2][1]])
        realPolyList.append(realPoly)
        dif = polyColourDict[tuple(sorted(poly, key=lambda x: (x[0], x[1])))][3]
        if dif < bestDif:
            bestDif = dif
        if dif > worstDif:
            worstPoly = poly
            worstRealPoly = realPoly
            worstDif = dif

    scaleFactor = worstDif / max(bestDif, 1)
    scaleFactor = max(min(scaleFactor, 4), 2)
    scaleFactor = 6 - scaleFactor
    if worstDif > 0:
        midpoint = ((worstRealPoly[0][0] + worstRealPoly[1][0] + worstRealPoly[2][0]) / 3, (worstRealPoly[0][1] + worstRealPoly[1][1] + worstRealPoly[2][1]) / 3)
        nodeList[x][y] = ((nodeList[x][y][0] + ((midpoint[0] - nodeList[x][y][0]) / scaleFactor)), (nodeList[x][y][1] + ((midpoint[1] - nodeList[x][y][1]) / scaleFactor)))


    for poly in polyList:
        polyColourDict[tuple(sorted(poly, key=lambda x: (x[0], x[1])))] = calculatePolyColour([nodeList[poly[0][0]][poly[0][1]], nodeList[poly[1][0]][poly[1][1]], nodeList[poly[2][0]][poly[2][1]]], pix)


#########################################################################################
##          Main start
#########################################################################################

targetImagePath = "temp.tif"
#if len(sys.argv) > 1:
   # targetImagePath = sys.argv[1]

targetImage = Image.open(targetImagePath)
pixels = targetImage.load()
WIDTH = targetImage.size[0]
HEIGHT = targetImage.size[1]

pix = []
for x in range(WIDTH):
    pixRow = []
    for y in range(HEIGHT):
        pixRow.append(pixels[x, y])
    pix.append(pixRow)

nodeList = []
polyColourDict = {}
for x in range(0, NUM_NODES_WIDE):
    nodeRow = []
    for y in range(0, NUM_NODES_HIGH):
        nodeRow.append((int(x * (WIDTH / float(NUM_NODES_WIDE - 1))), int(y * (HEIGHT / float(NUM_NODES_HIGH - 1)))))

    nodeList.append(nodeRow)

for x in range(0, NUM_NODES_WIDE - 1):
    for y in range(0, NUM_NODES_HIGH - 1):
        polyColourDict[tuple(sorted([(x, y), (x, y + 1), (x + 1, y + 1)], key=lambda x: (x[0], x[1])))] = calculatePolyColour([nodeList[x][y], nodeList[x][y + 1], nodeList[x + 1][y + 1]], pix)
        polyColourDict[tuple(sorted([(x, y), (x + 1, y), (x + 1, y + 1)], key=lambda x: (x[0], x[1])))] = calculatePolyColour([nodeList[x][y], nodeList[x + 1][y], nodeList[x + 1][y + 1]], pix)

pr = cProfile.Profile()
pr.enable()
count = 0
for i in range(0, 100):
    count = 0
    xList = list(range(1, NUM_NODES_HIGH - 1))
    random.shuffle(xList)
    yList = list(range(1, NUM_NODES_WIDE - 1))
    random.shuffle(yList)
    for y in xList:
        count += 1
        f = open("progress.txt", 'w')
        f.write(str(i) + ", " + str(count) + "\n")
        f.close()
        threadList = []
        for x in yList:
            moveNode(x, y, nodeList, polyColourDict, pix)

    img = Image.new('RGB', (WIDTH, HEIGHT))
    drw = ImageDraw.Draw(img, "RGB")
    
    drawNodeList(nodeList, polyColourDict, drw)

    outImagePath = targetImagePath.split(".")[0] + "_mesh.png"
    img.save(outImagePath)

pr.disable()

s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

print("total pixel in: " + str(total_pixel_in))
print("total pixel out: " + str(total_pixel_out))

FileNotFoundError: [Errno 2] No such file or directory: 'temp.tif'

In [3]:
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--shape-predictor", required=True,help="path to facial landmark predictor")
#ap.add_argument("-i", "--image", required=True,help="path to input image")
#args = vars(ap.parse_args())

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
#cap = cv2.VideoCapture('sgm.mp4')
#cap = cv2.VideoCapture(0)
# load the input image, resize it, and convert it to grayscale
while True:
    #ret, image = cap.read() 
    image = cv2.imread('gouse1.jpg')
    print(image.shape)
    image = imutils.resize(image, width=500)
    print(image.shape)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        #for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        name = 'jaw'
        (i, j) = face_utils.FACIAL_LANDMARKS_IDXS[name]
        (i, j) = (i+4,j-4)
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        #cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        
        for (x, y) in shape:
            cv2.circle(clone, (x, y), 3, (0, 255, 0), -1)

        # extract the ROI of the face region as a separate image
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        roi = image[y:y + h, x:x + w]
        roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)

        # show the particular face part
        #cv2.imshow("ROI", roi)
        cv2.imshow("Image", clone)
        #cv2.waitKey(0)
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    if cv2.waitKey(3000) & 0xFF == ord('q'):
        break

#cap.release() 
cv2.destroyAllWindows()

(486, 648, 3)
(375, 500, 3)
(486, 648, 3)
(375, 500, 3)


In [ ]:
import glob

import cv2

import numpy as np

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30 , 0.001)

objp = np.zeros((9*6,3), np.float32)

objp[:,:2] = np.mgrid[0:6.0:9].T.reshape(-1,2)

objpoints = []

imgpoints = []

images = glob.glob('./Muestras_Calibracio/*.jpg')

for fname in images:

  img = cv2.imread(fname)

  gray = cv2.cvtColor(img,cv.COLOR_BGR2GRAY)

  ret , corners = cv.findChessboardCorners(gray,(6,9), None)

  if ret == True:

    objpoints.append(objp)

    corners2 = cv2.cornerSubPix(gray , corners , (11,11) , (-1,-1) , criteria)

    imgpoints.append(corners)

    cv2.drawChessboardCorners(img , (6,9) , corners2 , ret)